In [89]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import rotations


In [90]:
def load_pickle(file_path):
    try:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [91]:
def discretize_action_to_control_mode_E2E(action):
    """
    -1 ~ 1 maps to 0 ~ 1
    """
    # Your action discretization logic here
    # print("Action: ", action)
    action_norm = (action + 1) / 2
    # print(action_norm, action)
    if 1 / 6 > action_norm >= 0:
        # print("| Slide up on right finger")
        control_mode = 0
        friction_state = 1  # left finger high friction
        pos_idx = 0
    elif 2 / 6 > action_norm >= 1 / 6:
        # print("| Slide down on right finger")
        control_mode = 1
        friction_state = 1
        pos_idx = 1
    elif 3 / 6 > action_norm >= 2 / 6:
        # print("| Slide up on left finger")
        control_mode = 2
        friction_state = -1
        pos_idx = 1
    elif 4 / 6 > action_norm >= 3 / 6:
        # print("| Slide down on left finger")
        control_mode = 3
        friction_state = -1
        pos_idx = 0
    elif 5 / 6 > action_norm >= 4 / 6:
        # print("| Rotate clockwise")
        control_mode = 4
        friction_state = 0
        pos_idx = 0
        # print("Rotate")
    else:
        assert 1 >= action_norm >= 5 / 6, f"Check: {action_norm}"
        # print("| Rotate anticlockwise")
        control_mode = 5
        friction_state = 0
        pos_idx = 1
        # print(pos_idx)
        # print("Rotate")
    return friction_state, control_mode, pos_idx

In [92]:
def compute_goal_radi(a, b, left_motor=[0.037012, -0.1845, 0.002], right_motor=[-0.037488, -0.1845, 0.002]):
    '''
    a is the left contact point,
    b is the right contact point
    '''
    # print("compute: ", a, b)
    a[2] = 0.002
    b[2] = 0.002

    # left_motor = [0.037012, -0.1845, 0.002]
    # right_motor = [-0.037488, -0.1845, 0.002]

    # print("Check radius: ", a, b, left_motor, right_motor)
    assert left_motor[2] == 0.002 and right_motor[2] == 0.002, f"Wrong motor height, check: {left_motor[2], right_motor[2]}"

    assert a.shape == b.shape
    assert a.shape[-1] == 3

    radius_al = np.zeros_like(a[..., 0])
    radius_br = np.zeros_like(b[..., 0])

    delta_r_a_left_motor = a[..., :3] - left_motor  # pos of motor left
    radius_al = np.linalg.norm(delta_r_a_left_motor, axis=-1)

    delta_r_b_right_motor = b[..., :3] - right_motor  # pos of motor right
    radius_br = np.linalg.norm(delta_r_b_right_motor, axis=-1)

    goal_radi = [radius_al,radius_br]

    return goal_radi

In [93]:
def compute_orientation_diff(goal_a, goal_b):
    ''' get pos difference and rotation difference
    left motor pos: 0.037012 -0.1845 0.002
    right motor pos: -0.037488 -0.1845 0.002
    '''
    assert goal_a.shape == goal_b.shape, f"Check: {goal_a.shape}, {goal_b.shape}"
    assert goal_a.shape[-1] == 7
    goal_a[2] = goal_b[2]

    d_pos = np.zeros_like(goal_a[..., 0])

    delta_pos = goal_a[..., :3] - goal_b[..., :3]
    d_pos = np.linalg.norm(delta_pos, axis=-1)

    quat_a, quat_b = goal_a[..., 3:], goal_b[..., 3:]

    euler_a = rotations.quat2euler(quat_a)
    euler_b = rotations.quat2euler(quat_b)
    if euler_a.ndim == 1:
        euler_a = euler_a[np.newaxis, :]  # Reshape 1D to 2D (1, 3)
    if euler_b.ndim == 1:
        euler_b = euler_b[np.newaxis, :]  # Reshape 1D to 2D (1, 3)
    euler_a[:, :2] = euler_b[:, :2]  # make the second and third term of euler angle the same
    quat_a = rotations.euler2quat(euler_a)
    quat_a = quat_a.reshape(quat_b.shape)

    # print(quat_a, quat_b)
    quat_diff = rotations.quat_mul(quat_a, rotations.quat_conjugate(quat_b))  # q_diff = q1 * q2*
    angle_diff = 2 * np.arccos(np.clip(quat_diff[..., 0], -1.0, 1.0))
    d_rot = angle_diff
    return d_pos, d_rot

In [94]:
def save_as_pickle(data, filename, directory='/Users/qiyangyan/Desktop/Diffusion/Demo_random'):
    os.makedirs(directory, exist_ok=True)
    full_path = os.path.join(directory, filename)

    try:
        with open(full_path, 'wb') as file:
            pickle.dump(data, file)
        # print(full_path)
        print(f"Data successfully saved to {full_path}")
    except Exception as e:
        print(f"An error occurred while saving data: {e}")

In [95]:
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-10000demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF-bigSteps-2000demos-randomise_1_repeat'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-testingDataset'
# data = load_pickle(pickle_file_path)

In [96]:
from collections import defaultdict

# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_0'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_2'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_bigSteps_20kDataset_withJointDR_seed_2'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_random_big_steps_cube_seed2'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demo_random/test_10k_cube_cylinder.pkl'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-10000demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demo_random/train_10k_three_cylinder_noObjVel.pkl'
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demo_random/train_10k_cube_cylinder_noObjVel.pkl'
data = load_pickle(pickle_file_path)

# pickle_file_path = [
#     '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_0',
#     '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_1',
#     '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_2'
# ]

# pickle_file_path = [
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_bigSteps_20kDataset_withJointDR_seed_0',
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_bigSteps_20kDataset_withJointDR_seed_1',
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_bigSteps_20kDataset_withJointDR_seed_2'
# ]

# pickle_file_path = [
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_random_big_steps_cube_seed0',
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_random_big_steps_cube_seed1',
#     '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF_random_big_steps_cube_seed2'
# ]

if len(np.shape(pickle_file_path)) == 0:
    data = load_pickle(pickle_file_path)
else:
    merged_dict = defaultdict(list)
    print(merged_dict)

    data_dicts = []
    for path in pickle_file_path:
        data_dicts.append(load_pickle(path))

    for d in data_dicts:
        for key, value in d.items():
            # Extend the list at the current key with the new values
            merged_dict[key].extend(value)

    data = dict(merged_dict)

In [97]:
print(data.keys())
for key in data.keys():
    print(np.shape(data[key]))

dict_keys(['observations', 'next_observations', 'desired_goals', 'sampled_desired_goals', 'actions', 'rewards', 'terminals'])
(77768, 18)
(77768, 24)
(77768, 11)
(77768, 11)
(77768, 2)
(0,)
(77768,)


In [98]:
count = 0
for item in data['terminals']:
    if item == 1:
        count += 1
print(count)

10000


In [99]:
# data['actions'] = [[action[0], action[1] * 2 - 1] for action in data['actions']]

In [100]:
count = 0
for item in data['terminals']:
    if item:
        count += 1
print(count)
print(data['terminals'][:20])

10000
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [101]:
# for i, terminal in enumerate(data['terminals']):
#     if terminal == 1:
#         data['actions'][i][1] = data['actions'][i][1] * 2 - 1

# Split demonstration into sliding and rotation

In [102]:
bigSteps_10000demos_rotation = {key: [] for key in data.keys()}
print(bigSteps_10000demos_rotation)
bigSteps_10000demos_slide = {key: [] for key in data.keys()}
print(bigSteps_10000demos_slide)

{'observations': [], 'next_observations': [], 'desired_goals': [], 'sampled_desired_goals': [], 'actions': [], 'rewards': [], 'terminals': []}
{'observations': [], 'next_observations': [], 'desired_goals': [], 'sampled_desired_goals': [], 'actions': [], 'rewards': [], 'terminals': []}


## Testing Data

In [103]:
# count = 0
# for i, action in enumerate(data['actions']):
#     control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
#     if control_mode == 5 or control_mode == 4:
#         count += 1
#         # if count > 10999:
#         if count > 999:
#             for key in data.keys():
#                 if len(data[key]) > i:
#                     if key == 'desired_goals':
#                         bigSteps_10000demos_rotation[key].append(data[key][i][:-2])
#                     else:
#                         bigSteps_10000demos_rotation[key].append(data[key][i])
#                     # bigSteps_10000demos_rotation[key].append(data[key][i])  # TODO: Uncomment this for 11 element 
#         bigSteps_10000demos_slide['terminals'][-1] = 1
#     else:
#         # if count > 10999:
#         if count > 999:
#             for key in data.keys():
#                 if len(data[key]) > i:
#                     if key == 'desired_goals':
#                         bigSteps_10000demos_slide[key].append(data[key][i][:-2])
#                     else:
#                         bigSteps_10000demos_slide[key].append(data[key][i])
#                     # bigSteps_10000demos_slide[key].append(data[key][i])  # TODO: Uncomment this for 11 element 
#         if data['terminals'][i] == 1:
#             count += 1
#             bigSteps_10000demos_slide['terminals'][-1] = 1
# 
# for key in bigSteps_10000demos_slide.keys():
#     bigSteps_10000demos_rotation[key] = np.array(bigSteps_10000demos_rotation[key])
#     bigSteps_10000demos_slide[key] = np.array(bigSteps_10000demos_slide[key])
# 
# print("| bigSteps_10000demos_rotation")
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# print("| bigSteps_10000demos_slide")
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))
# print(count)

In [104]:
# # save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_50kdemos_rotation_testingDataset")
# # save_as_pickle(bigSteps_10000demos_slide, "bigSteps_50kdemos_slide_testingDataset")
# 
# save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_50kdemos_rotation_withDR_testingDataset")
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_50kdemos_slide_withDR_testingDataset")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))

## Training data

In [105]:
print(sum(1 for item in data['terminals'] if item == 1))
for i, item in enumerate(data['terminals']):
    control_mode = discretize_action_to_control_mode_E2E(data['actions'][i][1])[1]
    if control_mode == 4 or control_mode == 5:
        assert item == 1, f"Check: {i}"

10000


In [106]:
count = 0
for i, action in enumerate(data['actions']):
    # if count >= 10000:
    #     break
    control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
    if control_mode == 5 or control_mode == 4:
        count += 1
        assert data['terminals'][i] == 1, f"check: {data['terminals'][i]}"
        for key in data.keys():
            if len(data[key]) > i:
                if key == 'desired_goals':
                    bigSteps_10000demos_rotation[key].append(data[key][i][:-2])
                else:
                    bigSteps_10000demos_rotation[key].append(data[key][i])
                # bigSteps_10000demos_rotation[key].append(data[key][i])  # TODO: Uncomment this for 11 element 
        # bigSteps_10000demos_slide['terminals'][-1] = 1

    else:
        for key in data.keys():
            if len(data[key]) > i:
                if key == 'desired_goals':
                    bigSteps_10000demos_slide[key].append(data[key][i][:-2])
                else:
                    bigSteps_10000demos_slide[key].append(data[key][i])
        if data['terminals'][i] == 1:
            count += 1
            # bigSteps_10000demos_slide['terminals'][-1] = 1
            # bigSteps_10000demos_slide[key].append(data[key][i])  # TODO: Uncomment this for 11 element 

for key in bigSteps_10000demos_slide.keys():
    bigSteps_10000demos_rotation[key] = np.array(bigSteps_10000demos_rotation[key])
    bigSteps_10000demos_slide[key] = np.array(bigSteps_10000demos_slide[key])

print(count)
count_1 = 0
for item in bigSteps_10000demos_slide['terminals']:
    if item == 1:
        count_1 += 1
print("Episodes in slide: ", count_1)

print("| bigSteps_10000demos_rotation")
for key in data:
    print(key, np.shape(bigSteps_10000demos_rotation[key]))

print("| bigSteps_10000demos_slide")
for key in data:
    print(key, np.shape(bigSteps_10000demos_slide[key]))

10000
Episodes in slide:  0
| bigSteps_10000demos_rotation
observations (10000, 18)
next_observations (10000, 24)
desired_goals (10000, 9)
sampled_desired_goals (10000, 11)
actions (10000, 2)
rewards (0,)
terminals (10000,)
| bigSteps_10000demos_slide
observations (67768, 18)
next_observations (67768, 24)
desired_goals (67768, 9)
sampled_desired_goals (67768, 11)
actions (67768, 2)
rewards (0,)
terminals (67768,)


In [107]:
count = 0
for i, item in enumerate(bigSteps_10000demos_slide['terminals']):
    if item == 1:
        count += 1
    if count == 999 and item == 1:
        print(i)
print(len(bigSteps_10000demos_slide['terminals']))
print(bigSteps_10000demos_slide['terminals'][3445:3448])
slide_test = {key: [] for key in bigSteps_10000demos_slide.keys()}
for key in data.keys():
    slide_test[key] = bigSteps_10000demos_slide[key][3445:]
for key in slide_test.keys():
    slide_test[key] = np.array(slide_test[key])
    print(np.shape(slide_test[key]))
# save_as_pickle(slide_test, "bigSteps_10k_demos_random_slide_testDataset")

67768
[0. 0. 0.]
(64323, 18)
(64323, 24)
(64323, 9)
(64323, 11)
(64323, 2)
(0,)
(64323,)


In [108]:
# print(action_1xn[:10])

# Save Files

In [109]:
# save_as_pickle(bigSteps_10000demos_rotation, "train_bigSteps_10k_three_cylinder_rotation_noObjVel")
# save_as_pickle(bigSteps_10000demos_slide, "train_bigSteps_10k_three_cylinder_slide_noObjVel")

save_as_pickle(bigSteps_10000demos_rotation, "train_bigSteps_10k_cube_cylinder_rotation_noObjVel")
save_as_pickle(bigSteps_10000demos_slide, "train_bigSteps_10k_cube_cylinder_slide_noObjVel")

print(pickle_file_path)

for key in data:
    print(key, np.shape(bigSteps_10000demos_rotation[key]))

for key in data:
    print(key, np.shape(bigSteps_10000demos_slide[key]))

Data successfully saved to /Users/qiyangyan/Desktop/Diffusion/Demo_random/train_bigSteps_10k_cube_cylinder_rotation_noObjVel
Data successfully saved to /Users/qiyangyan/Desktop/Diffusion/Demo_random/train_bigSteps_10k_cube_cylinder_slide_noObjVel
/Users/qiyangyan/Desktop/Diffusion/Demo_random/train_10k_cube_cylinder_noObjVel.pkl
observations (10000, 18)
next_observations (10000, 24)
desired_goals (10000, 9)
sampled_desired_goals (10000, 11)
actions (10000, 2)
rewards (0,)
terminals (10000,)
observations (67768, 18)
next_observations (67768, 24)
desired_goals (67768, 9)
sampled_desired_goals (67768, 11)
actions (67768, 2)
rewards (0,)
terminals (67768,)


In [110]:
# save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_10k_demos_random_rotation")
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_10k_demos_random_slide")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))


In [111]:
# save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_50kdemos_rotation_withJointDR")
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_50kdemos_slide_withJointDR")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))

# Add sliding end indicator


In [112]:
bigSteps_10000demos_slide_endIndicator = {key: [] for key in data.keys()}
print(bigSteps_10000demos_slide_endIndicator)

{'observations': [], 'next_observations': [], 'desired_goals': [], 'sampled_desired_goals': [], 'actions': [], 'rewards': [], 'terminals': []}


## Testing Dataset

In [113]:
# count = 0
# for i, action in enumerate(data['actions']):
#     control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
#     if control_mode == 4 or control_mode == 5:
#         count += 1
#         if count > 10999+1:
#             for key in data.keys():
#                 bigSteps_10000demos_slide_endIndicator['actions'][-1][-1] = 1
#     else:
#         if count > 10999:
#             for key in data.keys():
#                 if len(data[key]) > i:
#                     if key == 'desired_goals':
#                         bigSteps_10000demos_slide_endIndicator[key].append(data[key][i][:-2])  # remove the last two items
#                     elif key == 'actions':
#                         bigSteps_10000demos_slide_endIndicator[key].append(np.append(data[key][i], 0))  # add 0 to the end of action as an indicator
#                     else:
#                         # print(type(bigSteps_10000demos_slide_endIndicator[key]), key)
#                         bigSteps_10000demos_slide_endIndicator[key].append(data[key][i])
# 
# for key in bigSteps_10000demos_slide_endIndicator.keys():
#     bigSteps_10000demos_slide_endIndicator[key] = np.array(bigSteps_10000demos_slide_endIndicator[key])
# 
# print("| bigSteps_10000demos_with_indicator")
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide_endIndicator[key]))
# 
# print(count)

In [114]:
# save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_50kdemos_slide_endIndicator_withJointDR_testingDataset")
# # save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_50kdemos_slide_endIndicator_testingDataset")
# 
# print(pickle_file_path)
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))

## Training Dataset

In [115]:
count = 0
for item in data['terminals']:
    if item == 1:
        count += 1
print(count)

10000


In [116]:
count = 0
for i, action in enumerate(data['actions']):
    if count >= 50000:
        break
    control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
    if control_mode == 4 or control_mode == 5:
        count += 1
        for key in data.keys():
            bigSteps_10000demos_slide_endIndicator['actions'][-1][-1] = 1
    else:
        for key in data.keys():
            if len(data[key]) > i:
                if key == 'desired_goals':
                    bigSteps_10000demos_slide_endIndicator[key].append(data[key][i][:-2])  # remove the last two items
                elif key == 'actions':
                    bigSteps_10000demos_slide_endIndicator[key].append(np.append(data[key][i], 0))  # add 0 to the end of action as an indicator
                else:
                    # print(type(bigSteps_10000demos_slide_endIndicator[key]), key)
                    bigSteps_10000demos_slide_endIndicator[key].append(data[key][i])

for key in bigSteps_10000demos_slide_endIndicator.keys():
    bigSteps_10000demos_slide_endIndicator[key] = np.array(bigSteps_10000demos_slide_endIndicator[key])

print("| bigSteps_10000demos_with_indicator")
for key in data:
    print(key, np.shape(bigSteps_10000demos_slide_endIndicator[key]))

print(count)

| bigSteps_10000demos_with_indicator
observations (67768, 18)
next_observations (67768, 24)
desired_goals (67768, 9)
sampled_desired_goals (67768, 11)
actions (67768, 3)
rewards (0,)
terminals (67768,)
10000


In [117]:
# continue_action = bigSteps_10000demos_slide_endIndicator['actions'][:,2].reshape(-1, 1)
# print(bigSteps_10000demos_slide_endIndicator['actions'][:10])
# print(continue_action[:10])

In [118]:
# save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_10k_demos_random_slide_endIndicator")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))

In [119]:
# save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_50kdemos_slide_endIndicator_withJointDR")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))

# Visualization

In [120]:
# plt.figure(figsize=(12, 5))
# 
# action = np.array(bigSteps_10000demos_slide_endIndicator['actions'])
# true_action_first_elements = action[:, 0]
# true_action_second_elements = action[:, 1]
# true_action_third_elements = action[:, 2]
# 
# plt.subplot(1, 3, 1)  # First subplot
# # plt.plot(action_first_elements, label='True Action First Element', color='orange')
# # plt.plot(action_first_elements, label='Action First Element', linewidth=1, color='blue', linestyle='dashed')
# indices = list(range(len(true_action_first_elements)))  # Creating a list of indices
# plt.scatter(indices, true_action_first_elements, label='True Action First Element', color='orange',
#             s=10)  # Scatter plot
# 
# plt.title('First Element of Actions')
# # plt.ylim(-1, 1)
# plt.xlabel('Iteration')
# plt.ylabel('Value')
# plt.legend()
# 
# plt.subplot(1, 3, 2)  # Second subplot
# # plt.plot(action_second_elements, label='True Action Second Element', color='orange')
# indices = list(range(len(true_action_second_elements)))  # Creating a list of indices
# plt.scatter(indices, true_action_second_elements, label='True Action Second Element', color='orange',
#             s=5)  # Scatter plot
# 
# plt.title('Second Element of Actions')
# plt.xlabel('Iteration')
# plt.ylabel('Value')
# plt.legend()
# 
# plt.subplot(1, 3, 3)  # Second subplot
# # plt.plot(action_second_elements, label='True Action Second Element', color='orange')
# indices = list(range(len(true_action_third_elements)))  # Creating a list of indices
# plt.scatter(indices, true_action_third_elements, label='True Action Third Element', color='orange',
#             s=5)  # Scatter plot
# 
# plt.title('Third Element of Actions')
# plt.xlabel('Iteration')
# plt.ylabel('Value')
# plt.legend()

# Replace desired goal with slide achieved goal

In [121]:
bigSteps_10000demos_slideAG_as_DG = bigSteps_10000demos_slide.copy()
bigSteps_10000demos_middle_goal = []

In [122]:
count = 0
for i, action in enumerate(data['actions']):
    control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
    if control_mode == 4 or control_mode == 5:
        count += 1
        bigSteps_10000demos_middle_goal.append(data['observations'][i][8:15])

bigSteps_10000demos_middle_goal = np.array(bigSteps_10000demos_middle_goal)
print(np.shape(bigSteps_10000demos_middle_goal))
print("| bigSteps_10000demos_middle_goal")
print(bigSteps_10000demos_middle_goal[0])

(10000, 7)
| bigSteps_10000demos_middle_goal
[ 1.2116799e-02 -3.0413958e-01  4.9408904e-05  9.9869466e-01
  1.7801307e-03  7.6364013e-05  5.1047217e-02]


In [123]:
desired_goal_list_for_slide_demo = []
episode = 0
for i, action in enumerate(data['actions']):
    control_mode = discretize_action_to_control_mode_E2E(action[1])[1]
    if control_mode == 4 or control_mode == 5:
        episode += 1
    else:
        middle_goal_pos = bigSteps_10000demos_middle_goal[episode][:3]
        goal_radi = compute_goal_radi(middle_goal_pos, middle_goal_pos)  # TODO: This is for simulation
        ones = np.ones(2)
        desired_goal_list_for_slide_demo.append(np.concatenate([bigSteps_10000demos_middle_goal[episode], goal_radi, ones]))
        
print(episode)
print(len(desired_goal_list_for_slide_demo))
assert len(desired_goal_list_for_slide_demo) == len(bigSteps_10000demos_slide['actions'])

bigSteps_10000demos_slideAG_as_DG["desired_goals"] = np.array(desired_goal_list_for_slide_demo)
for key in data:
    print(key, np.shape(bigSteps_10000demos_slideAG_as_DG[key]))
print(bigSteps_10000demos_slideAG_as_DG["desired_goals"][-1])

10000
67768
observations (67768, 18)
next_observations (67768, 24)
desired_goals (67768, 11)
sampled_desired_goals (67768, 11)
actions (67768, 2)
rewards (0,)
terminals (67768,)
[ 3.40854265e-02 -2.93865919e-01  2.00000009e-03  9.87394512e-01
  4.57201786e-05  1.16020556e-05  1.58278331e-01  1.09405069e-01
  1.30704474e-01  1.00000000e+00  1.00000000e+00]


In [124]:
for i, obs in enumerate(bigSteps_10000demos_slideAG_as_DG['observations']):
    achieved_goal = obs[8:15]  # pose 7 elements
    desired_goal = bigSteps_10000demos_slideAG_as_DG['desired_goals'][i][:7]
    desired_goal_radi = bigSteps_10000demos_slideAG_as_DG['desired_goals'][i][7:9]
    assert len(achieved_goal) == len(desired_goal)
    _, angle_diff = compute_orientation_diff(np.array(desired_goal), np.array(achieved_goal))
    achieved_goal_radi = compute_goal_radi(achieved_goal[:3], achieved_goal[:3])
    radi_diff = desired_goal_radi - achieved_goal_radi
    
    bigSteps_10000demos_slideAG_as_DG['observations'][i][-3] = radi_diff[0]
    bigSteps_10000demos_slideAG_as_DG['observations'][i][-2] = radi_diff[1]
    bigSteps_10000demos_slideAG_as_DG['observations'][i][-1] = angle_diff

# Correct the mapping error in action for rotations

In [125]:
# for i, terminal in enumerate(data['terminals']):
#     if terminal == 1:
#         data['actions'][i][1] = data['actions'][i][1] * 2 - 1

# Uncomment the files you want to save

In [126]:
# save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_10000demos_slide_endIndicator")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))

In [127]:
# save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_10000demos_rotation")
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_10000demos_slide")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))


In [128]:
# save_as_pickle(bigSteps_10000demos_rotation, "bigSteps_10000demos_rotation_testingDataset")
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_10000demos_slide_testingDataset")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))


In [129]:
# save_as_pickle(bigSteps_10000demos_slide_endIndicator, "bigSteps_10000demos_slide_endIndicator_testingDataset")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_rotation[key]))

In [130]:
# save_as_pickle(bigSteps_10000demos_slide, "bigSteps_10000demos_slide_testingDataset_11elements")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slide[key]))

In [131]:
# save_as_pickle(bigSteps_10000demos_slideAG_as_DG, "bigSteps_10000demos_slideAG_as_DG")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slideAG_as_DG[key]))

In [132]:
# save_as_pickle(data, "/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10000demos_testingDataset")
# 
# for key in data:
#     print(key, np.shape(bigSteps_10000demos_slideAG_as_DG[key]))